# Text Processing

In [2]:
import numpy as np
import pandas as pd
from pymystem3 import Mystem
import re
import transformers
import nltk
from tqdm import notebook
import torch
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from dateutil.parser import parse
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForMaskedLM

## Данные

In [2]:
file_path = 'chd — 100.xlsx'

In [3]:
data = pd.read_excel(file_path)
data.head(1)

,admittion,department,discharge,sex,height,weight,BMI,BSA,birth,Операции (все в ИБ),...,Количество выб. из ОРИТ,К.дней в ОРИТ (всего),"ИВЛ, час. в ОРИТ (суммарно)",Инф. осложнения в ОРИТ,Назн. преп. в ОРИТ,target,Непосред. причина смерти,ЭхоКГ (Из Эпикр. вып.),ЭКГ (Из Эпикр. вып.),Назначения при выписке
0,2016-12-12,ehn,2017-01-10,m,76,9.7,111.27,0.46,02.01.2016,12.12.2016: (Откр./ИК) Перевязка ранее наложен...,...,2,20,252,01.01.2017: пневмония,"Адреналина г\хл 0,1% 1мл №5; Аксетин 750мг №10...",recovery,NaN,"ЭхоКГ ВПС (02.12.2016 14:37:22, врач Неталиева...",ЭКГ (02.12.2016 16:43:25)\nРитм сердца синусов...,


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 43 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   admittion                                        99 non-null     datetime64[ns]
 1   department                                       99 non-null     object        
 2   discharge                                        99 non-null     datetime64[ns]
 3   sex                                              99 non-null     object        
 4   height                                           99 non-null     int64         
 5   weight                                           99 non-null     float64       
 6   BMI                                              99 non-null     float64       
 7   BSA                                              99 non-null     float64       
 8   birth                                     

In [5]:
data.groupby('target')['target'].count()

target
death         5
no change     2
recovery     91
unrnown       1
Name: target, dtype: int64

Выкинем unkown, т.к. он всего один и для теста не нужен

In [6]:
data = data[data['target'] != 'unrnown']

Избавимся от дисбаланса классов оверсэмплингом и андерсэмплингом. Сразу выделим тестовую выборку

In [92]:
test_data = data[data['target'] == 'death'].head(2)
test_data = pd.concat([test_data, data[data['target'] == 'no change'].head(1)])
test_data = pd.concat([test_data, data[data['target'] == 'recovery'].head(10)])
test_data['target'].unique()

array(['death', 'no change', 'recovery'], dtype=object)

In [93]:
train_data = data[data['target'] == 'death'].tail(3)
train_data = pd.concat([train_data, data[data['target'] == 'no change'].tail(1)])
train_data = pd.concat([train_data, data[data['target'] == 'recovery'].tail(81)])
train_data['target'].unique()

array(['death', 'no change', 'recovery'], dtype=object)

In [94]:
c1 = train_data[train_data['target'] == 'death']
c2 = train_data[train_data['target'] == 'no change']
c3 = train_data[train_data['target'] == 'recovery']

train_data = pd.concat([c3.sample(50), c2.sample(25, replace=True), c1.sample(25, replace=True)]).reset_index()
train_data['target'].value_counts()

recovery     50
no change    25
death        25
Name: target, dtype: int64

Для теста возьмем только один столбец - диагноз

In [95]:
diagnosis = train_data['Диагноз']

In [96]:
diagnosis[1]

'Тетрада Фалло; Комбинированый стеноз легочной артерии; Открытое овальное окно; Операция 16.09.2016 наложение модифицированного подключично-легочного анастомоза по Blalock слева c синтетическим протезом из PTFE; Операция 13.04.2017 перевязка ранее наложенного анастомоза по Blalock c синтетическим протезом слева; радикальная коррекция тетрады Фалло с пластикой выводного отдела правого желудочка и ствола легочной артерии ксеноперикардиальной заплатой'

## Пример применения TF-IDF на столбце "Диагноз"

### Гипотетический способ отчистки конкретных элементов текста (например даты)

In [63]:
nDAY = r'(?:[0-3]?\d)'
nMNTH = r'(?:11|12|10|0?[1-9])' 
nYR = r'(?:(?:19|20)\d\d)'
nDELIM = r'(?:[\/\-\._])?'

In [64]:
re.sub(f'(?:{nDAY}{nDELIM}{nMNTH}{nDELIM}{nYR})', ' ', diagnosis[1])

'Атрезия легочной артерии 1 тип ; Перимембранозный приточный дефект межжелудочковой перегороки (некоммитированный); Открытое овальное окно. Стеноз устья левой легочной артерии, умеренная гипоплазия левой легочной артерии; Дискордантное отхождение аорты от правого желудочка; Состояние после операции ( ) наложение модифицированного подключично-легочного анастомоза по Blalock справа; Большая аорто-легочная коллатеральная артерия к правому легкому; Операция ( ) перевязка ранее наложенного анастомоза по Blalock справа; реконструкция путей оттока из правого желудочка кондуитом из яремной вены быка, в условиях ИК и гипотермии; Недостаточность кровообращения 2-б степени'

### Лемментизация

In [16]:
m = Mystem()

# Леммантизирует текст
def lemmatize(text):
    return "".join(m.lemmatize(text))

In [17]:
lemmatize(diagnosis[1])

'общий открытый атриовентрикулярный канал тип а по растелль. дефект межпредсердный перегородка вторичный. недостаточность митральный компонент 2 степень. недостаточность трикуспидальный компонент 2-3 степень; аномальный мышца в полость правый желудочек. состояние после операция от от 06.10.2016. манжет на легочный артерия. НК 2А-1 степень; операция 19.05.2017 г. атриовентрикулярный блокада 1 степень. миграция водитель ритм по предсердие. НК 1 степень\n'

### Отчистка текста

In [18]:
# Чистит текст от всего, кроме русских и английских букв
def clear_text(text):
    cleaned = re.sub(r'[^а-яА-Яa-zA-ZёЁ ]', ' ', text)
    cleaned = cleaned.split()
    return ' '.join(cleaned)

In [19]:
clear_text(lemmatize(diagnosis[1]))

'общий открытый атриовентрикулярный канал тип а по растелль дефект межпредсердный перегородка вторичный недостаточность митральный компонент степень недостаточность трикуспидальный компонент степень аномальный мышца в полость правый желудочек состояние после операция от от манжет на легочный артерия НК А степень операция г атриовентрикулярный блокада степень миграция водитель ритм по предсердие НК степень'

### TF-IDF

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Стивен\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [97]:
stopwords = set(nltk_stopwords.words('russian'))

In [98]:
%%time

corpus = diagnosis.apply(lambda x: clear_text(lemmatize(x)))

CPU times: total: 188 ms
Wall time: 1min 17s


In [99]:
%%time

corpus_test = test_data['Диагноз'].apply(lambda x: clear_text(lemmatize(x)))

CPU times: total: 0 ns
Wall time: 10.2 s


In [110]:
tf_idf = TfidfVectorizer(stop_words=stopwords).fit(corpus)

In [111]:
tf_idf_train = tf_idf.transform(corpus)
tf_idf_test = tf_idf.transform(corpus_test)

In [113]:
tf_idf_train.toarray().shape

(100, 364)

In [114]:
tf_idf_test.toarray().shape

(13, 364)

## RuBert

In [125]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruBert-base")

model = AutoModelForMaskedLM.from_pretrained("sberbank-ai/ruBert-base")

Downloading:   0%|          | 0.00/683M [00:00<?, ?B/s]

Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [126]:
tokenized = corpus.apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

In [135]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
max_len

194

In [138]:
padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)
padded[0]

array([   101,    116,   2991,  74639, 113404,    666,    107,  30545,
        12866,    378,  31338,    390,   1996,  89679,  20900,    378,
        71738,   5635,  62761,    701,  10429,    391,    378,  39381,
          657,   3526,  91552,    378,  73147,    667,    378,  36986,
          396,  30545,  12866,    378,  31338,    390,  61896,  52065,
         6056,  91706,  20900,    378,  73147,  11894, 113404,  91645,
         1277,  12866,    378,  22389,   2154,  20900,    378,  31338,
          390,    700,  80548,    755,    378, 113404,    666,    114,
         4984,   2160,  12020,   5762,    934,  11803,  45165,   9309,
        41156,    667,    378,  30545,  12866,    378,   3904,   2968,
         4211,  13474,    108,    385,    106,  11894,  99540,    667,
          378,  31338,    922,    667,    378,  31053,  69367,   5917,
        11803,  71738,   5635,   8838,   3624,  33988,    378,  39381,
          657,  83047,    660,  30545,  12866,    378,  31338,    390,
      

In [149]:
batch_size = 10
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    
    print(f'Stage {i}')
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    
    embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

Stage 0
Stage 1
Stage 2
Stage 3
Stage 4
Stage 5
Stage 6
Stage 7
Stage 8
Stage 9


In [151]:
features = np.concatenate(embeddings)

In [154]:
features.shape

(100, 120138)

Получилось 120000 признаков. **Надо использовать маленькую библиотеку слов, желательно из медицинских терминов.**

## Тест модели

In [155]:
X, y = features, data['target']

In [1]:
X_train, X_test, y_train, y_test = (pd.DataFrame(tf_idf_train.toarray()),
                                    pd.DataFrame(tf_idf_test.toarray()),
                                    train_data['target'],
                                    test_data['target']
                                   )

NameError: name 'pd' is not defined

In [156]:
model = CatBoostClassifier(loss_function='MultiClass', verbose=100)
model = model.fit(X_train, y_train, eval_set=(X_test, y_test))

Learning rate set to 0.103116
0:	learn: 0.9795262	test: 1.0289566	best: 1.0289566 (0)	total: 3.57ms	remaining: 3.57s
100:	learn: 0.0218449	test: 0.9338064	best: 0.7276733 (21)	total: 299ms	remaining: 2.66s
200:	learn: 0.0098701	test: 1.0450191	best: 0.7276733 (21)	total: 613ms	remaining: 2.44s
300:	learn: 0.0065405	test: 1.1114453	best: 0.7276733 (21)	total: 917ms	remaining: 2.13s
400:	learn: 0.0048288	test: 1.1654236	best: 0.7276733 (21)	total: 1.24s	remaining: 1.85s
500:	learn: 0.0038256	test: 1.2095427	best: 0.7276733 (21)	total: 1.54s	remaining: 1.54s
600:	learn: 0.0031713	test: 1.2453865	best: 0.7276733 (21)	total: 1.85s	remaining: 1.23s
700:	learn: 0.0027177	test: 1.2757372	best: 0.7276733 (21)	total: 2.16s	remaining: 920ms
800:	learn: 0.0023712	test: 1.3023782	best: 0.7276733 (21)	total: 2.47s	remaining: 613ms
900:	learn: 0.0021059	test: 1.3262626	best: 0.7276733 (21)	total: 2.77s	remaining: 305ms
999:	learn: 0.0018935	test: 1.3474340	best: 0.7276733 (21)	total: 3.08s	remaining:

In [117]:
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

       death       0.00      0.00      0.00         2
   no change       0.00      0.00      0.00         1
    recovery       0.77      1.00      0.87        10

    accuracy                           0.77        13
   macro avg       0.26      0.33      0.29        13
weighted avg       0.59      0.77      0.67        13



C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [121]:
dummy_model = DummyClassifier().fit(X_train, y_train)
print(classification_report(y_test, dummy_model.predict(X_test)))

              precision    recall  f1-score   support

       death       0.00      0.00      0.00         2
   no change       0.00      0.00      0.00         1
    recovery       0.77      1.00      0.87        10

    accuracy                           0.77        13
   macro avg       0.26      0.33      0.29        13
weighted avg       0.59      0.77      0.67        13



C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [122]:
tree_model = DecisionTreeClassifier().fit(X_train, y_train)
print(classification_report(y_test, tree_model.predict(X_test)))

              precision    recall  f1-score   support

       death       1.00      0.50      0.67         2
   no change       0.00      0.00      0.00         1
    recovery       0.83      1.00      0.91        10

    accuracy                           0.85        13
   macro avg       0.61      0.50      0.53        13
weighted avg       0.79      0.85      0.80        13



C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
